In [1]:
from __future__ import division
from __future__ import print_function
import time
import random
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from utils import *
from model import *
import uuid
import os
from tensorboardX import SummaryWriter

# Training settings
import easydict
args = easydict.EasyDict({
    "seed": 50,
    "epochs": 200,
    "lr": 0.01,
    "wd1": 0.01,
    "wd2": 5e-4,
    "layer": 32,
    "hidden": 64,
    "dropout": 0.6,
    "patience": 100,
    "data": 'cora',
    "dev": 0,
    "alpha": 0.1,
    "lamda": 0.5,
    "variant": False,
    "test": True
})

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

# Load data
adj, features, labels,idx_train,idx_val,idx_test = load_citation(args.data)
cudaid = "cuda:"+str(args.dev)
device = torch.device(cudaid)
features = features.to(device)
adj = adj.to(device)
checkpt_file = 'pretrained/'+uuid.uuid4().hex+'.pt'
print(cudaid,checkpt_file)

model = GCNII(nfeat=features.shape[1],
                nlayers=args.layer,
                nhidden=args.hidden,
                nclass=int(labels.max()) + 1,
                dropout=args.dropout,
                lamda = args.lamda, 
                alpha=args.alpha,
                variant=args.variant,
                 pair='PN').to(device)

optimizer = optim.Adam([
                        {'params':model.params1,'weight_decay':args.wd1},
                        {'params':model.params2,'weight_decay':args.wd2},
                        ],lr=args.lr)

def train():
    model.train()
    optimizer.zero_grad()
    output = model(features,adj)
    
    acc_train = accuracy(output[idx_train], labels[idx_train].to(device))
    loss_train = F.nll_loss(output[idx_train], labels[idx_train].to(device))
    loss_train.backward()
    optimizer.step()   
    
    return loss_train.item(),acc_train.item()


def validate():
    model.eval()
    with torch.no_grad():
        output = model(features,adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val].to(device))
        acc_val = accuracy(output[idx_val], labels[idx_val].to(device))
        return loss_val.item(),acc_val.item()

def test():
    model.load_state_dict(torch.load(checkpt_file))
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_test = F.nll_loss(output[idx_test], labels[idx_test].to(device))
        acc_test = accuracy(output[idx_test], labels[idx_test].to(device))
        return loss_test.item(),acc_test.item()
result = {}    
t_total = time.time()
bad_counter = 0
best = 999999999
best_epoch = 0
acc = 0
for epoch in range(args.epochs):
    loss_tra,acc_tra = train()
    loss_val,acc_val = validate()

    if(epoch+1)%10 == 0: 
        print('Epoch:{:04d}'.format(epoch+1),
            'train',
            'loss:{:.3f}'.format(loss_tra),
            'acc:{:.2f}'.format(acc_tra*100),
            '| val',
            'loss:{:.3f}'.format(loss_val),
            'acc:{:.2f}'.format(acc_val*100))
  
    if loss_val < best:
        best = loss_val
        best_epoch = epoch
        acc = acc_val
        torch.save(model.state_dict(), checkpt_file)
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == args.patience:
        break

if args.test:
    acc = test()[1]

print("Train cost: {:.4f}s".format(time.time() - t_total))
print('Load {}th epoch'.format(best_epoch))
print("Test" if args.test else "Val","acc.:{:.2f}".format(acc*100))

cuda:0 pretrained/021e5ff276774e0cbcff8553fb8d5be6.pt
 pair ===== 33333333 PN
Epoch:0010 train loss:1.867 acc:47.86 | val loss:1.855 acc:72.20
Epoch:0020 train loss:1.703 acc:64.29 | val loss:1.650 acc:75.80
Epoch:0030 train loss:1.445 acc:60.71 | val loss:1.375 acc:78.00
Epoch:0040 train loss:1.177 acc:74.29 | val loss:1.140 acc:78.60
Epoch:0050 train loss:1.028 acc:76.43 | val loss:0.993 acc:77.80
Epoch:0060 train loss:0.978 acc:70.71 | val loss:0.861 acc:79.20
Epoch:0070 train loss:0.928 acc:76.43 | val loss:0.810 acc:79.40
Epoch:0080 train loss:0.787 acc:78.57 | val loss:0.774 acc:79.40
Epoch:0090 train loss:0.771 acc:82.14 | val loss:0.749 acc:78.60
Epoch:0100 train loss:0.821 acc:79.29 | val loss:0.735 acc:79.40
Epoch:0110 train loss:0.730 acc:82.86 | val loss:0.704 acc:79.80
Epoch:0120 train loss:0.646 acc:82.14 | val loss:0.712 acc:79.00
Epoch:0130 train loss:0.658 acc:77.86 | val loss:0.705 acc:79.60
Epoch:0140 train loss:0.672 acc:83.57 | val loss:0.682 acc:79.40
Epoch:0150 t

In [2]:


# Training settings
import easydict
args = easydict.EasyDict({
    "seed": 38,
    "epochs": 200,
    "lr": 0.01,
    "wd1": 0.01,
    "wd2": 5e-4,
    "layer": 64,
    "hidden": 64,
    "dropout": 0.6,
    "patience": 100,
    "data": 'citeseer',
    "dev": 0,
    "alpha": 0.1,
    "lamda": 0.5,
    "variant": False,
    "test": True
})

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

# Load data
adj, features, labels,idx_train,idx_val,idx_test = load_citation(args.data)
cudaid = "cuda:"+str(args.dev)
device = torch.device(cudaid)
features = features.to(device)
adj = adj.to(device)
checkpt_file = 'pretrained/'+uuid.uuid4().hex+'.pt'
print(cudaid,checkpt_file)

model = GCNII(nfeat=features.shape[1],
                nlayers=args.layer,
                nhidden=args.hidden,
                nclass=int(labels.max()) + 1,
                dropout=args.dropout,
                lamda = args.lamda, 
                alpha=args.alpha,
                variant=args.variant,
                 pair='').to(device)

optimizer = optim.Adam([
                        {'params':model.params1,'weight_decay':args.wd1},
                        {'params':model.params2,'weight_decay':args.wd2},
                        ],lr=args.lr)

def train():
    model.train()
    optimizer.zero_grad()
    output = model(features,adj)
    
    acc_train = accuracy(output[idx_train], labels[idx_train].to(device))
    loss_train = F.nll_loss(output[idx_train], labels[idx_train].to(device))
    loss_train.backward()
    optimizer.step()   
    
    return loss_train.item(),acc_train.item()


def validate():
    model.eval()
    with torch.no_grad():
        output = model(features,adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val].to(device))
        acc_val = accuracy(output[idx_val], labels[idx_val].to(device))
        return loss_val.item(),acc_val.item()

def test():
    model.load_state_dict(torch.load(checkpt_file))
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_test = F.nll_loss(output[idx_test], labels[idx_test].to(device))
        acc_test = accuracy(output[idx_test], labels[idx_test].to(device))
        return loss_test.item(),acc_test.item()
result = {}    
t_total = time.time()
bad_counter = 0
best = 999999999
best_epoch = 0
acc = 0
for epoch in range(args.epochs):
    loss_tra,acc_tra = train()
    loss_val,acc_val = validate()

    if(epoch+1)%20 == 0: 
        print('Epoch:{:04d}'.format(epoch+1),
            'train',
            'loss:{:.3f}'.format(loss_tra),
            'acc:{:.6f}'.format(acc_tra*100),
            '| val',
            'loss:{:.3f}'.format(loss_val),
            'acc:{:.6f}'.format(acc_val*100))
  
    if loss_val < best:
        best = loss_val
        best_epoch = epoch
        acc = acc_val
        torch.save(model.state_dict(), checkpt_file)
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == args.patience:
        break

if args.test:
    acc = test()[1]

print("Train cost: {:.4f}s".format(time.time() - t_total))
print('Load {}th epoch'.format(best_epoch))
print("Test" if args.test else "Val","acc.:{:.2f}".format(acc*100))

cuda:0 pretrained/d03a65c31d53475eb9ed961cd2a001e1.pt
 pair ===== 33333333 


ModuleAttributeError: 'GCNII' object has no attribute 'norm'